In [ ]:
#pip install bs4
#pip install requests

Parsing part of project

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

HEADERS = {'User-Agent': 'Mozilla/5.0'}

response = requests.get('https://en.wikipedia.org/wiki/List_of_highest-grossing_films', headers= HEADERS)

soup = BeautifulSoup(response.content, "lxml")

table = soup.find('table')
films = []


for row in table.select('tr')[1:]:
    film = {}
    if not row.find('a').text in ['RK', 'TS3']:
        film['name']  = row.find('a').text
        film['href'] = row.find('a')['href'] 
        
    else:
        film['name']  = row.find_all('a')[1].text
        film['href'] = row.find_all('a')[1]['href'] 
        
        
    film['Box Office Revenue']  = row.find_all('td')[2].text[:-1].replace(',', '')
    film['Box Office Revenue'] = film['Box Office Revenue'] [film['Box Office Revenue'].find('$'):]
    film['year']  = row.find_all('td')[3].text[:-1]   
    films.append(film)
    

for film in films:
    url = "https://en.wikipedia.org" + film['href']
    #print(url)
    #print(url)
    response = requests.get(url, headers = HEADERS)
    soup = BeautifulSoup(response.content, "lxml")
    author = soup.find('th', text = 'Directed by')
    country = soup.find('th', text = 'Country')
    country = soup.find('th', text = 'Countries') if country == None else country
    table = soup.select_one('table')
    for tr in soup.find_all('tr'):
        if author in tr:
            #print(tr.find('td').find('a').text, tr.find('td').find('ul'))
            if (tr.find('td').find('ul') == None):
                film['Director'] = tr.find('td').find('a').text
            else:
                film['Director'] = []
                for li in tr.find('td').find('ul').find_all('li'):
                    film['Director'].append(li.text)
            
        if country in tr:
            if (tr.find('td').find('ul') == None):
                film['Country'] = tr.find('td').text
            else:
                film['Country'] = []
                for li in tr.find('td').find('ul').find_all('li'):
                    film['Country'].append(li.text[:-3])
            break
            
films

Database part

In [1]:
!pip install sqlalchemy

In [7]:
from sqlalchemy import create_engine

engine = create_engine('sqlite+pysqlite:///:memory:', echo=True)

In [8]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
#create a new declarative class that maps to a database table

C:\Users\Nikita\AppData\Local\Temp\ipykernel_8716\2750559068.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [9]:
from sqlalchemy import Column, Integer, String

class Films(Base):
    __tablename__ = 'films'
    id = Column(Integer, primary_key=True, autoincrement= True)
    title = Column(String, nullable=False)
    release_year = Column(Integer)
    director = Column(String)
    box_office = Column(String)
    country = Column(String)
    
    


In [10]:
Base.metadata.create_all(engine)

2025-02-25 21:43:00,403 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 21:43:00,405 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("films")
2025-02-25 21:43:00,406 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-25 21:43:00,408 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("films")
2025-02-25 21:43:00,409 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-25 21:43:00,411 INFO sqlalchemy.engine.Engine 
CREATE TABLE films (
	id INTEGER NOT NULL, 
	title VARCHAR NOT NULL, 
	release_year INTEGER, 
	director VARCHAR, 
	box_office VARCHAR, 
	country VARCHAR, 
	PRIMARY KEY (id)
)


2025-02-25 21:43:00,412 INFO sqlalchemy.engine.Engine [no key 0.00064s] ()
2025-02-25 21:43:00,413 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
import json

def preprocess(films_raw):    
    films = []
    
    for film_raw in films_raw:
        film = dict()
        film['title'] = film_raw['name']
        film['release_year'] = int(film_raw['year'])
        if isinstance(film_raw['Director'] ,list):
            film['director'] = ', '.join(director for director in film_raw['Director'] ) 
        else :
            film['director'] = film_raw['Director']
        film['box_office'] = film_raw['Box Office Revenue'][1:]
        if isinstance(film_raw['Country'], list)  :
            film['country'] = ', '.join(country for  country in film_raw['Country'])
        else:
            film['country'] = film_raw['Country']
        films.append(film)
    
    return films


 

In [17]:
'''
file = open('filmsFinal.json', 'w')
json.dump({'films' : films})    
films_rows = [Films(**f) for f in films]
    
json.dump(films_rows, file)
file.close()
f1.close()
'''  

from  sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

session.query(Films).delete()

films_rows = preprocess(films)
films_rows = [Films(**f) for f in films_rows ]
session.add_all(films_rows)

session.commit()

2025-02-25 21:49:09,692 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-25 21:49:09,695 INFO sqlalchemy.engine.Engine DELETE FROM films
2025-02-25 21:49:09,696 INFO sqlalchemy.engine.Engine [cached since 232.8s ago] ()
2025-02-25 21:49:09,714 INFO sqlalchemy.engine.Engine INSERT INTO films (title, release_year, director, box_office, country) VALUES (?, ?, ?, ?, ?) RETURNING id
2025-02-25 21:49:09,716 INFO sqlalchemy.engine.Engine [generated in 0.00075s (insertmanyvalues) 1/50 (ordered; batch not supported)] ('Avatar', 2009, 'James Cameron', '2923706026', 'United Kingdom, United States')
2025-02-25 21:49:09,720 INFO sqlalchemy.engine.Engine INSERT INTO films (title, release_year, director, box_office, country) VALUES (?, ?, ?, ?, ?) RETURNING id
2025-02-25 21:49:09,723 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/50 (ordered; batch not supported)] ('Avengers: Endgame', 2019, 'Anthony Russo, Joe Russo', '2797501328', 'United States')
2025-02-25 21:49:09,725 INFO sqlalchemy.en

In [23]:
films = session.query(Films)

C:\Users\Nikita\AppData\Local\Temp\ipykernel_8716\2290887910.py:2: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  films.get(2).__dict__


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x216e920e680>,
 'director': 'Anthony Russo, Joe Russo',
 'id': 2,
 'country': 'United States',
 'release_year': 2019,
 'box_office': '2797501328',
 'title': 'Avengers: Endgame'}

In [25]:
res = {'films' : []}
for f in films:
    f = f.__dict__
    res['films'].append({
        'director' : f['director'],
        'country' : f['country'],
        'release_year' : f['release_year'],
        'box_office' : f['box_office'],
        'title' : f['title']
        })


2025-02-25 21:59:46,391 INFO sqlalchemy.engine.Engine SELECT films.id AS films_id, films.title AS films_title, films.release_year AS films_release_year, films.director AS films_director, films.box_office AS films_box_office, films.country AS films_country 
FROM films
2025-02-25 21:59:46,393 INFO sqlalchemy.engine.Engine [cached since 548.3s ago] ()


In [ ]:
f = open('filmsFinal.json', 'w')

json.dump()